## Problem biznesowy  

**Opis zadania**: “Nasz serwis zarabia wtedy, gdy ludzie słuchają muzyki. Jeśli nie wiemy ile czasu będą słuchali jej w przyszłości, to trudno nam rozliczać się z artystami i negocjować z nimi stawki”

### definicja problemu biznesowego
1. Obecna sytuacja: Nie umiemy oszacować czasu słuchania artystów, a tym samym zysków, które przyniosą naszemu serwisowi. Utrudnia to rozliczanie i negocjowanie stawek.
2. Chcemy wprowadzić system szacowania czasu słuchania poszczególnych artystów, który:
    - analizując dane zebrane na temat artystów będzie szacował czas ich słuchania przez użytkowników
    - będzie przedstawiał tę informację w formie czytelnej dla człowkieka, np. w postaci krótkiego tekstu/tabelki

### definicja zadania/zadań modelowania
- Należy zaprojektować model predykcyjny, aproksymujący czas słuchania danego artysty przez użytkowników na podstawie danych o artystach, użytownikach i słuchanych przez nich utworach
- Dane do modelowania:
    - Dane o artystach (nazwa, gatunek muzyczny)
    - Dane o użytkownikach (chętnie słuchane utwory, ulubione gatunki muzyczne, ulubieni artyści)
    - Dane o utworach (częstość i daty przesłuchań, czas trwania, liczba polubień/pominięć, gatunek)

### definicja wszystkich założeń
- Model będzie realizować predykcje w skali miesiąca (na miesiąc w przód)
- Zadanie będzie realizowane przez model regresji
- Model będzie trenowany na dostępnych danych offline

### zaproponowane kryteria sukcesu
- Biznesowe: Różnica w przewidywaniach modelu, a faktycznym czasem słuchania danego artysty nie przekracza 20% w stosunku do rzeczywistej wartości
- Analityczne: Takie samo, jak biznesowe osiągnięte na danych offline


## Uwagi do danych   
1. Brak opisu co oznaczają poszczególne atrybuty i co zawierają poszczególne pliki
    - Nie wiemy czy poszczególne atrybuty ciągłe, dyskretne itp. 
    - Czy mamy informacje o zakresach wartości?
2. Dużo błędów w danych - przykład: znaki typu \0 w imionach/nazwach, -1 w id artystów
3. Braki w danych - wartości typu null  
4. Bardzo mało danych dotyczących użytkowników w porównaniu do pozostałych



**Potrzebne pakiety**

In [105]:
import pandas as pd

### Artists

In [106]:
jsonl_file_path_artists = 'data/IUM23Z_Zad_06_01_v1/artists.jsonl'
df_artists = pd.read_json(jsonl_file_path_artists, lines=True)
print("Ogólny opis danych:\n", "\t liczba pól:", len(df_artists), "\n", df_artists.describe(), "\n")
print("Duplikaty id:\n", df_artists["id"][df_artists["id"].duplicated(keep=False)].value_counts(), "\n")
print("Liczba pól name zawierających kodowanie binarne (\\u):", df_artists['name'][df_artists['name'].apply(lambda x: any(ord(char) > 127 for char in x))].count(), "\n")
print("Liczba pustych pól genres: ", df_artists["genres"].isnull().sum(), "\n")


Ogólny opis danych:
 	 liczba pól: 27650 
            id   name              genres
count   27650  27650               22085
unique  22042  27542               11582
top        -1    TNT  [classic thai pop]
freq     5609      4                  61 

Duplikaty id:
 id
-1    5609
Name: count, dtype: int64 

Liczba pól name zawierających kodowanie binarne (\u): 3063 

Liczba pustych pól genres:  5565 



**Uwagi i problemy**:
- id
    - Duża liczba niewłaściwych pól = -1 (aż 20%)
    - Pozostałe wartości są unikalne, brak pustych pól
- name
    - Dużo błędów w nazwach artystów - 11% zawiera kodowanie binarne
    - Nie wszystkie pola są unikalne - około 30 wartości się powtarza maksymalnie 4 razy
- genres
    - 20% pól jest pustych
    - niektóre gatunki są rozdzielane przecinkami, inne nie - następuje pewna niespójność
    - pytanie czy nie lepiej zastosować kodowanie wskaźnikowe? tzn. utworzyć atrybuty typu _classic, thai, pop_ o kodowaniu binarnym (0/1)
    
### Sessions

In [107]:
jsonl_file_path_sessions = 'data/IUM23Z_Zad_06_01_v1/sessions.jsonl'
df_sessions = pd.read_json(jsonl_file_path_sessions, lines=True)
print("Ogólny opis danych:\n", "\t liczba pól:", len(df_sessions), "\n")
print("Ogólny opis atrybutu timestamp:\n", df_sessions["timestamp"].describe(), "\n")
print("Liczba pustych pól user_id: ", df_sessions["user_id"].isnull().sum(), "\n")
print("Wartości przybierane przez user_id: \n", df_sessions["user_id"].unique(), "\n")
cleaned_df_sessions = df_sessions["user_id"].dropna().astype(str)
print("Najczęściej przybierane 3 wartości przez user_id: \n", cleaned_df_sessions.value_counts()[:3], "\n")
print("Najrzadziej przybierane 3 wartości przez user_id: \n", cleaned_df_sessions.value_counts()[-3:], "\n")
print("Liczba pustych pól track_id (null i pustych stringów): ", df_sessions["track_id"].isnull().sum() + (df_sessions["track_id"] == "").sum(), "\n")
cleaned_df_sessions = df_sessions["track_id"][df_sessions["track_id"] != ""].dropna()
print("Najczęściej przybierane 3 wartości przez track_id: \n", cleaned_df_sessions.value_counts()[:3], "\n")
print("Najrzadziej przybierane 3 wartości przez track_id: \n", cleaned_df_sessions.value_counts()[-3:], "\n")
print("Wartości przybierane przez event_type: \n", df_sessions["event_type"].value_counts(dropna=False), "\n")
session_description = df_sessions["session_id"].describe()
print(f"Ogólny opis session_id:\n liczba niepustych pól:{int(session_description['count'])}, minimum: {int(session_description['min'])}, maksimum: {int(session_description['max'])}, liczba unikalnych wartości: {df_sessions['session_id'].nunique()}")


Ogólny opis danych:
 	 liczba pól: 82763 

Ogólny opis atrybutu timestamp:
 count                            82763
mean     2023-07-23 16:21:52.495853056
min         2023-01-06 07:18:18.388696
25%      2023-05-22 17:40:49.987560960
50%      2023-08-01 22:29:45.408012032
75%      2023-09-30 15:25:12.824009984
max         2023-11-19 23:57:36.314518
Name: timestamp, dtype: object 

Liczba pustych pól user_id:  16577 

Wartości przybierane przez user_id: 
 [101.  nan 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127.
 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141.
 142. 143. 144. 145. 146. 147. 148. 149. 150.] 

Najczęściej przybierane 3 wartości przez user_id: 
 user_id
105.0    2498
136.0    2488
143.0    2393
Name: count, dtype: int64 

Najrzadziej przybierane 3 wartości przez user_id: 
 user_id
133.0    170
104.0    130
135.0    104
Name: count, dtype: int64 

Liczba pustych pól track

**Uwagi i problemy**
- timestamp
    - brak błędów i braków w danych
    - dużo mniej danych z pierwszych 7 miesięcy niż z ostatnich 3
- user_id
    - 20% pól jest pustych 
    - dość nierównomiernie rozłożone - ponad 2tys danych dla niektórych użytkowników, mniej niż 200 dla innych
- track_id
    - 20% pól jest pustych
- event_type
    - brak błędów w danych
    - 20% pól jest pustych
- session_id
    - brak błędów i braków w danych


### Track storage

In [108]:
jsonl_file_path_track_storage = 'data/IUM23Z_Zad_06_01_v1/track_storage.jsonl'
df_track_storage = pd.read_json(jsonl_file_path_track_storage, lines=True)
print("Ogólny opis danych:\n", "\t liczba pól:", len(df_track_storage), "\n")
print("Liczba niepustych pól track_id: ", df_track_storage["track_id"].count(), "\n")
print("Ogólny opis storage_class: \n", df_track_storage["storage_class"].describe(), df_track_storage["storage_class"].value_counts(), "\n")
print("Ogólny opis daily_cost: \n", df_track_storage["daily_cost"].describe(), "\n")

Ogólny opis danych:
 	 liczba pól: 129648 

Liczba niepustych pól track_id:  129648 

Ogólny opis storage_class: 
 count     129648
unique         3
top         slow
freq      128433
Name: storage_class, dtype: object storage_class
slow      128433
medium      1208
fast           7
Name: count, dtype: int64 

Ogólny opis daily_cost: 
 count    129648.000000
mean          0.011514
std           0.005612
min           0.000200
25%           0.008887
50%           0.010885
75%           0.013253
max           0.201381
Name: daily_cost, dtype: float64 



Dane nie wydają się być potrzebne do naszego zadania - pomijam opis

### Tracks


In [109]:
jsonl_file_path_tracks = 'data/IUM23Z_Zad_06_01_v1/tracks.jsonl'
df_tracks = pd.read_json(jsonl_file_path_tracks, lines=True)
print("Ogólny opis danych:\n", "\t liczba pól:", len(df_tracks), "\n")
for column in df_tracks.columns:
    print(f"Ogólny opis {column}: \n", df_tracks[column].describe(), "\nLiczba pustych pól:", df_tracks[column].isnull().sum(),"\n")

print("Liczba pól name zawierających kodowanie binarne (\\u):", df_tracks['name'][df_tracks['name'].apply(lambda x: any(ord(char) > 127 for char in x)if pd.notna(x) else False)].count(), "\n")

Ogólny opis danych:
 	 liczba pól: 129648 

Ogólny opis id: 
 count                     103615
unique                    103615
top       4q7EBRq8ncztC2PRmEC7Fy
freq                           1
Name: id, dtype: object 
Liczba pustych pól: 26033 

Ogólny opis name: 
 count         103862
unique         93340
top       Summertime
freq              23
Name: name, dtype: object 
Liczba pustych pól: 25786 

Ogólny opis popularity: 
 count    103826.000000
mean         29.670429
std          17.089842
min           0.000000
25%          17.000000
50%          29.000000
75%          41.000000
max          97.000000
Name: popularity, dtype: float64 
Liczba pustych pól: 25822 

Ogólny opis duration_ms: 
 count    1.296480e+05
mean     2.281700e+05
std      1.093728e+05
min      4.000000e+03
25%      1.774000e+05
50%      2.167870e+05
75%      2.632670e+05
max      4.027622e+06
Name: duration_ms, dtype: float64 
Liczba pustych pól: 0 

Ogólny opis explicit: 
 count    129648.000000
mean         

**Uwagi i problemy**
- id
    - 20% pól pustych
    - reszta pól unikalna
- name
    - 20% pól pustych
    - 19% zawiera kodowanie binarne
- popularity, duration_ms, explicit i release_date
    - brak pustych pól i błędów
- id_artist
    - 20% pól pustych 
Reszta pól nie wydaje się być potrzebna - pomijam

### Users

In [110]:
jsonl_file_path_users = 'data/IUM23Z_Zad_06_01_v1/users.jsonl'
df_users = pd.read_json(jsonl_file_path_users, lines=True)
print("Ogólny opis danych:\n", "\t liczba pól:", len(df_users), "\n")
for column in df_users.columns:
    print(f"Ogólny opis {column}: \n", "Liczba pustych pól:", df_users[column].isnull().sum())
    if column in ["name", "city", "street"]:
        print("Liczba pól zawierających kodowanie binarne (\\u):", df_users[column][df_users[column].apply(lambda x: any(ord(char) > 127 for char in x))].count())
    print("\n")


Ogólny opis danych:
 	 liczba pól: 50 

Ogólny opis user_id: 
 Liczba pustych pól: 0


Ogólny opis name: 
 Liczba pustych pól: 0
Liczba pól zawierających kodowanie binarne (\u): 18


Ogólny opis city: 
 Liczba pustych pól: 0
Liczba pól zawierających kodowanie binarne (\u): 20


Ogólny opis street: 
 Liczba pustych pól: 0
Liczba pól zawierających kodowanie binarne (\u): 11


Ogólny opis favourite_genres: 
 Liczba pustych pól: 14


Ogólny opis premium_user: 
 Liczba pustych pól: 6


Ogólny opis id: 
 Liczba pustych pól: 43




**Uwagi i problemy**
- id
    - zupełnie przypadkowy i niepotrzebny atrybut
- favourite_genres
    - 14 z 50 pól pustych

reszta pól raczej się nie przyda - pomijam